[Data Source](https://www.dshs.texas.gov/immunizations/data/school/coverage)

[tx_counties.geojson](https://github.com/Cincome/tx.geojson/blob/master/counties/tx_counties.geojson)

The dataset contains **vaccination** coverage **rates** (as proportions) for **Texas seventh graders by ISD** (Independent School District) from **2019–2024** school years. Key fields appear to include:
  * **ISD Name**
  * **Facility Address**
  * **County**
  * Coverage rates for vaccines: **Tdap/Td**, **Meningococcal**, **Hepatitis A**, **Hepatitis B**, **MMR**, **Polio**, **Varicella**

## Our goals with these datasets are:
1.	**Descriptive Statistics & Distributions**
  * **Compute mean**, median, variance of each vaccine’s coverage rate.
  * **Plot histograms** or box-plots to compare distribution shapes (e.g. is Varicella uptake more spread out than MMR?).

2.	**Group Comparisons**
  * By School Type (Public vs Private): test for significant differences in coverage with t-tests or non-parametric tests.
  * By County: rank counties by average coverage for each vaccine, identify top-/bottom-performers.
	
3.	**Temporal Trends**
  * Track how coverage rates change from 2019–2024 for each vaccine (line plots).
  * Compute year-over-year deltas and highlight vaccines with improving or declining uptake.

4.	**Correlation & Multivariate Relationships**
  * Correlate coverage rates across vaccines (e.g. do schools with high Tdap also have high MMR?).
  * If you re-merge in the exemption rates, explore negative correlations between exemptions and coverage.

5.	**Clustering & Segmentation**
  * **Cluster ISDs** or counties by their multi-vaccine profile using k-means or hierarchical clustering.
  * **Visualize clusters** on a map or in a PCA plot to find patterns.

6.	**Geospatial Mapping**
  * Use a **choropleth of Texas counties** to show geographic variations in vaccine uptake.
  * Overlay school-type or exemption cluster boundaries.

7.	**Anomaly/Outlier Detection**
  * Identify schools or years where coverage suddenly drops or spikes, flagging data-quality issues or local events.

8.	**Predictive Modeling & Forecasting**
  * Fit time series models (ARIMA, Prophet) per vaccine to forecast next year’s coverage.
  * Build regression models to predict coverage rates from demographic or exemption covariates.

9.	**Threshold & Compliance Monitoring**
  * **Tabulate** how many ISDs fall below critical public-health thresholds (e.g. MMR < 95%).
  * **Automatically** flag these for follow-up.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Because we have several datasets, we need to concatenate them into one
import pandas as pd

In [ ]:
# We define a mapping from school year to file path
files = {
    "2019-2020": "2019-2020-School-Vaccination-Coverage-Levels-by-District-Private-School-and-County---Seventh-Grade.xls",
    "2020-2021": "2020-2021-School-Vaccination-Coverage-Levels-by-District-Private-School-and-County---Seventh-Grade.xlsx",
    "2021-2022": "2021-2022-School-Vaccination-Coverage-by-District-and-County-Seventh-Grade.xls",
    "2022-2023": "2022-2023-School-Vaccination-Coverage-by-District-and-County-Seventh-Grade.xlsx",
    "2023-2024": "2023-2024_School_Vaccination_Coverage_Levels_Seventh_Grade.xlsx",
}

# Then, we read each file, tag with its school year, and collect into a list
coverage_dfs = []

for year, path in files.items():
    df = pd.read_excel(path, header=2)  
    df["School Year"] = year

    coverage_dfs.append(df)

# Then we concatenate into a single DataFrame
combined_coverage = pd.concat(coverage_dfs, ignore_index=True)

# We standardize column names
combined_coverage.columns = (
     combined_coverage.columns
     .str.strip()
     .str.replace(" ", "_")
     .str.lower()
 )

# Then, we inspect the result
print("Combined shape:", combined_coverage.shape)
print(combined_coverage.head())

# Then we save to a new csv file
combined_coverage.to_csv("merged_seventh_grade_vaccination_2019_2024.csv", index=False)
print("Merged dataset saved to merged_seventh_grade_vaccination_2019_2024.csv")

In [ ]:
df = pd.read_csv("merged_seventh_grade_vaccination_2019_2024.csv")
df.info()

In [ ]:
# We need to drop the last column in the DataFrame because it was a duplicate County column with null values
df.drop(columns=df.columns[-1], inplace=True)
print("Columns after dropping the last one:", df.columns.tolist())

In [ ]:
df.info()
df.head()

## Coverage Distribution

In [ ]:
import plotly.express as px

# Descriptive Statistics & Distributions

In [ ]:
# We need to double check our data for any duplicated column and drop that bad boy
df.dropna(axis=1, how='all', inplace=True)
for col in df.select_dtypes(include=['object']).columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.shape

In [ ]:
# We need to identify coverage-rate columns
def is_coverage_col(s: pd.Series) -> bool:
    s = s.dropna()
    return pd.api.types.is_numeric_dtype(s) and s.between(0, 1).all()

coverage_cols = [c for c in df.columns if is_coverage_col(df[c])]
coverage_cols

In [ ]:
# We need to compute mean, median, variance
stats_df = pd.DataFrame({
    'Vaccine': coverage_cols,
    'Mean': df[coverage_cols].mean().values,
    'Median': df[coverage_cols].median().values,
    'Variance': df[coverage_cols].var().values
})
stats_df

In [ ]:
# Then, we melt for plotting
plot_df = stats_df.melt(
    id_vars='Vaccine',
    value_vars=['Mean', 'Median', 'Variance'],
    var_name='Metric',
    value_name='Value'
)
plot_df.head()

In [ ]:
# Then we need to plot our interactive bar chart
fig = px.bar(
    plot_df,
    x='Vaccine',
    y='Value',
    color='Metric',
    barmode='group',
    title='Mean, Median, and Variance of Vaccine Coverage Rates',
    labels={'Value':'Statistic Value', 'Vaccine':'Vaccine'}
)
fig.update_layout(
    yaxis_title='Statistic Value',
    xaxis_tickangle=-45,
    height=500
)
fig.show()

In [ ]:
# We nee to prepare the subset of vaccine columns
vax_cols = ['tdap/td', 'meningococcal', 'hepatitis_a', 
            'hepatitis_b', 'mmr', 'polio', 'varicella']
df_subset = df[vax_cols].dropna()

# Then, we melt into long form
df_long = df_subset.melt(var_name='Vaccine', value_name='Coverage')

# Next, we overlay histogram
fig = px.histogram(
    df_long,
    x='Coverage',
    color='Vaccine',
    barmode='overlay',
    nbins=20,
    opacity=0.6,
    title='Coverage Distribution Comparison: All Vaccines'
)
fig.update_layout(
    xaxis_title='Coverage Rate',
    yaxis_title='Count of Records',
    xaxis=dict(range=[0, 1]),
    legend_title_text='Vaccine'
)
fig.show()

The overlaid histograms let us compare, at a glance, how the seven vaccines’ coverage rates are distributed across the schools and years:
**Interpreting the Overlay**
  * **Sharp Peaks** Near **1.0**:  **MMR**, **Polio**, **Tdap/Td**: These vaccines show most schools clustered at very high coverage, indicating uniformly strong compliance.
  * **Broader Distributions** **Hepatitis A**, **Hepatitis B**, **Varicella**: These curves spread further into lower‐coverage bins (**0.7–0.95**), revealing greater variability—some districts lag behind the top tier.
  * **Meningococcal** often sits in between: many districts have high uptake, but you’ll still see a visible tail of moderate coverage rates.

# Group Comparisons

In [ ]:
import scipy.stats as stats

In [ ]:
# We are re-importing the dataset due to the data prep we performed above
df = pd.read_csv("merged_seventh_grade_vaccination_2019_2024.csv")

# Here, we define vaccine columns
vax_cols = ['tdap/td', 'meningococcal', 'hepatitis_a',
            'hepatitis_b', 'mmr', 'polio', 'varicella']
vax_cols = [c for c in vax_cols if c in df.columns]

# Then, we coerce vaccine columns to numeric
for col in vax_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Next, we identify school types
types = df['school_type'].dropna().unique().tolist()
print("School types detected:", types)

# Then, we prepare group-by subset
grp = df.dropna(subset=['school_type'])

# And we perform ANOVA and Kruskal-Wallis
results = []
for col in vax_cols:
    # And build arrays per school_type
    data_groups = [grp.loc[grp['school_type'] == t, col].dropna() for t in types]
    
    # We need to only include types with at least 2 samples
    data_groups = [g for g in data_groups if len(g) >= 2]
    if len(data_groups) < 2:
        continue
    
    # This is the ANOVA grouping
    f_stat, f_p = stats.f_oneway(*data_groups)
    
    # This is the Kruskal-Wallis grouping
    h_stat, h_p = stats.kruskal(*data_groups)
    
    # Here, we are computing the means
    means = {f"{t}_mean": grp.loc[grp['school_type'] == t, col].dropna().mean() for t in types}
    
    # Then, we append
    res = {
        'Vaccine': col,
        'ANOVA_F': f_stat, 'ANOVA_p': f_p,
        'Kruskal_H': h_stat, 'Kruskal_p': h_p,
        **means
    }
    results.append(res)

results_df = pd.DataFrame(results)

# We then display our results

display(results_df.style.format({ 'ANOVA_F':'{:.3f}','ANOVA_p':'{:.3f}','Kruskal_H':'{:.3f}','Kruskal_p':'{:.3f}' }))

# Finally, we box plots our results for visual comparison
long_df = df.melt(
    id_vars='school_type',
    value_vars=vax_cols,
    var_name='Vaccine',
    value_name='Coverage'
).dropna()

fig = px.box(
    long_df,
    x='Vaccine',
    y='Coverage',
    color='school_type',
    title='Vaccine Coverage by School Type',
    labels={'school_type': 'School Type', 'Coverage': 'Coverage Rate'}
)
fig.update_layout(xaxis_tickangle=-45, yaxis_range=[0, 1], height=500)
fig.show()

### Group Comparison Analysis of Vaccine Coverage by School Type

#### Identify Groups
We extract distinct categories from the `school_type` column (e.g., Public School, Private School, Public ISD).  
These categories serve as our comparison groups.

#### Compute Group Means
For each vaccine (Tdap/Td, Meningococcal, Hepatitis A, Hepatitis B, MMR, Polio, Varicella), we calculate the **average coverage rate** in each school type.  
This reveals which school types tend to have higher or lower uptake.

#### One-Way ANOVA
- **Objective:** Test if the mean coverage differs across school types.  
- **Null Hypothesis (H₀):** All group means are equal.  
- **F-Statistic:** Ratio of between-group to within-group variance.  
- **p-Value:** Probability of observing the F-statistic under H₀.  
  - A small p (e.g., < 0.05) indicates significant differences in means.

#### Kruskal–Wallis Test
- **Purpose:** Nonparametric alternative to ANOVA for non‑Normal or heteroscedastic data.  
- **H-Statistic:** Based on ranked data across groups.  
- **p-Value:** Tests if at least one group’s median differs.  
  - A small p-value indicates a significant difference in distributions.

#### Visualization
We use **box plots** to display each vaccine’s coverage distribution by school type:  
- **Median** and **interquartile range** show central tendency and spread.  
- **Outliers** highlight exceptional values.  
This visual comparison complements statistical test results.

#### Interpreting Our Results
- **Group Means:** Indicate which school types have higher or lower average coverage.  
- **Significant ANOVA p-value:** Implies at least one pair of school types differs in mean coverage.  
  - Follow-up with post‑hoc tests (e.g., Tukey’s HSD) to pinpoint specific differences.  
- **Significant Kruskal p-value:** Confirms distributional differences without assuming Normality.  
- **Box Plots:** Allow quick assessment of overlap and variation among school types.

In [ ]:
import folium
import json
from folium.features import GeoJsonTooltip

# We load our GeoJSON
with open("tx_counties.geojson") as f:
    geo = json.load(f)

# Here, we're peeking the feature’s properties
print(geo["features"][0]["properties"].keys())
print(geo["features"][12]["properties"])

In [ ]:
# Here, we are preparing our data for vaccines observation.
vax_cols = ['tdap/td','meningococcal','hepatitis_a','hepatitis_b','mmr','polio','varicella']
coverage_cols = [c for c in vax_cols if c in df.columns]
for c in coverage_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')
df['geo_county'] = df['county'].astype(str).apply(
    lambda x: x if x.endswith("County") else f"{x} County"
)
county_stats = df.groupby('geo_county', as_index=False)[coverage_cols].mean()

# Then, we base our map centered on Texas
m = folium.Map(location=[31.0, -100.0], zoom_start=6)

# Next, we add a choropleth for each vaccine directly to the Map
geojson_path = "tx_counties.geojson"
for vaccine in coverage_cols:
    chor = folium.Choropleth(
        geo_data=geojson_path,
        name=f"{vaccine.title()} Coverage",
        data=county_stats,
        columns=["geo_county", vaccine],
        key_on="feature.properties.COUNTY",
        fill_color="YlGn",
        fill_opacity=0.6,
        line_opacity=0.2,
        legend_name=f"Avg {vaccine.title()} Coverage",
        overlay=True,
        control=True
    )
    chor.add_to(m)

# fill_color options: YlOrRd, YlGn, YlGnBu, BuGn, OrRd, Reds

# Here, we then overlay county labels with tooltips
folium.GeoJson(
    geojson_path,
    name="County Labels",
    style_function=lambda feat: {"fillOpacity": 0, "weight": 0.5, "color": "gray"},
    tooltip=GeoJsonTooltip(fields=["COUNTY"], aliases=["County:"])
).add_to(m)

# We set our layer control
folium.LayerControl(collapsed=False).add_to(m)

# Finally,, we save and display our interactive follium map
m.save("tx_vaccine_coverage_layers.html")

display(m)

### In the above GeoJSON map, we can in the Vaccine Choropleth Map that:

The average vaccination coverage rates for seven vaccines across Texas counties using an interactive Folium map with multiple overlay layers.

#### Data Preparation
- **Load merged dataset** combining seven-grade vaccination coverage (2019–2024).
- **Clean data**: drop fully null columns, coerce coverage columns to numeric, and normalize county names by appending "County" where needed.

#### County-Level Aggregation
- **Group by county** (`geo_county`) to compute **mean coverage** for each vaccine:
  - Tdap/Td
  - Meningococcal
  - Hepatitis A
  - Hepatitis B
  - MMR
  - Polio
  - Varicella

#### Base Map Setup
- **Initialize** a Folium Map centered on Texas (latitude 31.0, longitude -100.0, zoom level 6).

#### Overlay Choropleth Layers
- For each vaccine, **add a Chromopleth layer**:
  - **Data source**: county-level average coverage.
  - **GeoJSON**: Texas counties boundary file.
  - **Key**: `feature.properties.COUNTY` matches each county polygon.
  - **Style**: `YlGn` color scale with adjustable opacity.
  - **Toggleable**: layers can be turned on/off via the control panel.

#### County Labels
- **Add** a GeoJson layer with transparent fill and a tooltip showing the county name on hover.

#### Layer Control
- **Enable** Folium’s LayerControl to toggle each vaccine coverage layer and the county labels.

#### Output
- **Save** the interactive map as `tx_vaccine_coverage_layers.html`.
  * **Interact** with the map in any browser to explore vaccine coverage patterns by county.
- **Display** we can interact with the GeoJSON map in the Jupyter notebook as well.

## Temporal Trends

In [ ]:
# Then, we compute mean coverage by school year
yearly = (
    df
    .groupby("school_year", as_index=False)[coverage_cols]
    .mean()
)

# Next, we melt for plotting
yearly_long = yearly.melt(
    id_vars="school_year",
    value_vars=coverage_cols,
    var_name="Vaccine",
    value_name="Avg_Coverage"
)

# Then,we set our interactive line plot
fig = px.line(
    yearly_long,
    x="school_year",
    y="Avg_Coverage",
    color="Vaccine",
    markers=True,
    title="Temporal Trends: Vaccine Coverage Rates (2019–2024)",
    labels={"school_year":"School Year","Avg_Coverage":"Average Coverage"}
)
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis=dict(range=[0.9,1], title="Coverage Rate"),
    legend_title="Vaccine",
    height=500
)
fig.show()

**Interactive Line Plot**  
  * **X-Axis**: **school_year** (categorical axis showing each year from 2019–2024)
  * **Y-Axis**: **Avg_Coverage** (the mean coverage rate, between 0 and 1)
  * **Color**: Each vaccine gets its own colored line.
  * **Markers**: Small dots at each year help you pinpoint exact values.
  * **Interactivity**: Hover over any point to see the precise average coverage, zoom in on certain years, or toggle vaccines on/off in the legend.

**Interpreting the Trends**  
  *  **Rising Lines**: A vaccine whose line slopes upward indicates improving uptake over time.
  *  **Flat Lines**: Little change year-to-year suggests stable coverage—either consistently high or low.
  *  **Dip or Peak**: A dip in a particular year may signal local policy changes or supply issues; a peak could reflect successful campaigns.
  *  **Relative Comparison**: By seeing all vaccines on the same plot, you can compare which ones improve fastest or lag behind.

**Example Insights**  
  * If **MMR** starts at **0.95** in **2019** and climbs to **0.98** by **2024**, that shows near‐universal uptake becoming even more universal.
  * If Varicella hovers around **0.85** with minor fluctuations, it highlights where targeted interventions might be needed.
  * Divergences between vaccines can reveal where specific outreach or policy efforts succeeded (or stalled).

In [ ]:
# Here, we compute average coverage by school year
year_labels = ["2019-2020","2020-2021","2021-2022","2022-2023","2023-2024"]
yearly = (
    df
    .groupby("school_year", as_index=False)[coverage_cols]
    .mean()
)
# Then, we ensure correct order
yearly["school_year"] = pd.Categorical(yearly["school_year"],
                                       categories=year_labels,
                                       ordered=True)
yearly = yearly.sort_values("school_year")

# Next, we compute year-over-year deltas
delta = yearly.set_index("school_year")[coverage_cols].diff().reset_index()

# Then, we melt for plotting
delta_long = (
    delta
    .melt(id_vars="school_year",
          value_vars=coverage_cols,
          var_name="Vaccine",
          value_name="Delta")
    .dropna(subset=["Delta"])
)

# Then, we add our directional flag
delta_long["Direction"] = delta_long["Delta"].apply(
    lambda x: "Improving" if x>0 else ("Declining" if x<0 else "No Change")
)

# Next, we set the interactive facet bar chart
fig = px.bar(
    delta_long,
    x="school_year",
    y="Delta",
    color="Direction",
    facet_col="Vaccine",
    facet_col_wrap=3,
    category_orders={"school_year": year_labels, "Direction": ["Improving","Declining","No Change"]},
    color_discrete_map={"Improving":"green","Declining":"red","No Change":"purple"},
    title="Year-over-Year Δ in Vaccine Coverage (2019–2024)",
    labels={"Delta":"Δ Coverage Rate","school_year":"School Year"}
)
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_title="Year-over-Year Δ",
    height=800
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1].title()))
fig.show()

### In the above chart, we interpret:  
1.	**Aggregates mean** coverage per vaccine by school year.
2.	**Computes the difference** (diff()) between consecutive years for each vaccine.
3.	**Flags each Δ as “Improving”** (> 0), “Declining” (< 0), or “No Change” (= 0).
4.	**Plots a small-multiples** bar chart (one facet per vaccine) showing the year-over-year Δ, colored by direction.

## Correlation & Multivariate Relationships

In [ ]:
# We compute the Pearson correlation matrix
corr = df[coverage_cols].corr()

# Then, we display the numeric matrix
print("Correlation matrix of vaccine coverage rates:")
print(corr.round(3))

# Next, we plot an interactive heatmap
fig = px.imshow(
    corr,
    x=coverage_cols,
    y=coverage_cols,
    color_continuous_scale="RdBu",
    zmin=-1, zmax=1,
    title="Correlation Matrix of Vaccine Coverage Rates",
    labels={"color":"Pearson r"}
)
fig.update_layout(
    xaxis_tickangle=-45,
    width=600,
    height=600
)
fig.show()

**What We Computed**  
  * Pearson correlation coefficients between every pair of vaccine coverage rates (e.g. **Tdap/Td vs MMR**).
  * A correlation r ranges from –1 (perfect inverse) through 0 (no linear relationship) to +1 (perfect positive).
  * We restricted to columns whose values lie between 0 and 1 (true coverage rates), and then built the correlation matrix.

**Reading the Matrix**  
  * Diagonal elements are always 1 (each vaccine perfectly correlates with itself).
  * Off-diagonal elements tell you how two different vaccines move together:
    * **r > 0.8**: very strong positive relationship—schools that are good at one vaccine tend to be good at the other.
    * **r ≈ 0.5–0.8**: moderate positive relationship.
    * **r ≈ 0.2–0.5**: weak positive.
    * **r ≈ 0**: little to no linear association.

**Visualizing with a Heatmap**  
  * **The interactive heatmap** colors cells red for strong positive, blue for strong negative (rare here), and white near zero.
  * Hover over any cell to see the exact r value.

**Example Insights**  
  * If **Tdap/Td** and **MMR** show **r ≈ 0.95**, it suggests that districts achieving high compliance with Tdap almost always achieve high MMR coverage too.
  * A slightly lower correlation (say **r ≈ 0.75**) between Hepatitis A and Varicella might indicate that uptake strategies or exemptions differ for those vaccines.
  * Any unexpectedly low correlation (e.g. **r < 0.5**) flags a pair worth deeper investigation—perhaps logistical issues or policy differences affect one vaccine but not another.

**Why It Matters**  
  * **Program alignment**: Strong correlations suggest efficient, bundled delivery of multiple vaccines.
  * **Targeted interventions**: Weaker correlations identify vaccines that lag behind peers, guiding where to focus outreach or policy changes.
  * **Policy evaluation**: Tracking these correlations over time can show whether integrated vaccination efforts are becoming more or less cohesive.

## Clustering & Segmentation

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# We re-load dataset
df = pd.read_csv("merged_seventh_grade_vaccination_2019_2024.csv")

In [ ]:
df.dropna(axis=1, how='all', inplace=True)

# Then, we define expected vaccine columns
vax_cols = ['tdap/td', 'meningococcal', 'hepatitis_a',
            'hepatitis_b', 'mmr', 'polio', 'varicella']
coverage_cols = [c for c in vax_cols if c in df.columns]

# Next, we coerce to numeric
for c in coverage_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# And then, we group by county
county_stats = df.groupby("county", as_index=False)[coverage_cols].mean()
county_stats.dropna(subset=coverage_cols, inplace=True)

if not coverage_cols:
    raise ValueError("No vaccine coverage columns found. Please check column names.")

# Then, we standardize our features
scaler = StandardScaler()
X = scaler.fit_transform(county_stats[coverage_cols])

# We define the K-Means clustering
k = 4
kmeans = KMeans(n_clusters=k, random_state=42)
county_stats["cluster"] = kmeans.fit_predict(X)

# We now set the cluster sizes
cluster_sizes = county_stats["cluster"].value_counts().sort_index()
display(cluster_sizes.rename_axis("cluster").reset_index(name="count"))

# Then, we set the PCA for visualization
pca = PCA(n_components=2, random_state=42)
pcs = pca.fit_transform(X)
pc_df = pd.DataFrame(pcs, columns=["PC1", "PC2"])
pc_df["county"] = county_stats["county"]
pc_df["cluster"] = county_stats["cluster"]

fig = px.scatter(
    pc_df, x="PC1", y="PC2", color="cluster", hover_data=["county"],
    title=f"KMeans (k={k}) Clusters of Counties by Vaccine Coverage (PCA)"
)
fig.show()

# Next, we cluster centers in original scale
orig_centers = scaler.inverse_transform(kmeans.cluster_centers_)
centers_df = pd.DataFrame(orig_centers, columns=coverage_cols)
centers_df["cluster"] = range(k)
centers_melt = centers_df.melt(id_vars="cluster", var_name="Vaccine", value_name="Avg Coverage")

fig2 = px.bar(
    centers_melt, x="Vaccine", y="Avg Coverage", color="cluster", barmode="group",
    title="Cluster Centers: Average Coverage by Vaccine"
)
fig2.show()

# Gaines County

## Per our analysis, we clearly see Gaines County have a major vaccination issue

### In the above chart, we can make the below explaination:

**Feature Construction**  
    We start with each county’s mean coverage rate for each of the seven vaccines (Tdap/Td, Meningococcal, Hep A, Hep B, MMR, Polio, Varicella). This turns each county into a seven-dimensional “profile” of vaccine uptake.

**Standardization**  
    Because coverage rates for different vaccines can have different scales or variances, we **z-score** each dimension (subtract its mean and divide by its standard deviation). This ensures that no single vaccine dominates the clustering just by having a wider spread.

**K-Means Clustering**  
    We choose **K=4** (but you could experiment with other values) and let **K-Means** partition the counties into four groups based on their standardized multi-vaccine profiles.  
    * Cluster assignment: each county is labeled **0–3** according to which of the four centroids it’s nearest to in **7-D** space.
    * Cluster sizes: we print how many counties fell into each group so you can see if clusters are roughly balanced or skewed.

**PCA 2D Visualization**  
    To make these **high-dimensional** clusters visible, we run a Principal Components Analysis **(PCA)** and project each county onto the first two principal components (the two directions that capture the most variation).  
    * We then plot each county as a point in this **2D PCA** plane, colored by cluster label.
    * County names appear on hover, so you can identify examples from each group.
    * Well-separated clouds of points indicate that the clustering found genuinely distinct vaccine-uptake patterns.

**Cluster Centers Profile**  
    Finally, we un-standardize each cluster’s centroid back into the original coverage-rate scale and plot a grouped bar chart showing the average coverage for each vaccine in each cluster. This lets you read off, for example:  
    * **Cluster 0** might have uniformly high **(> 95%)** coverage across all vaccines—“top performers.”
    * **Cluster 1** might score high on MMR and Polio but lag on Varicella—suggesting targeted outreach needs.
    * **Cluster 2** might be low across the board—counties requiring broad program support.
    * **Cluster 3** might present a mixed profile (e.g. strong Hep B but weak Tdap).

**Why It’s Useful**  
  * **Segmentation**: Rather than treating every county identically, you discover natural groupings and can tailor interventions to each cluster’s strengths and weaknesses.
  * **Resource targeting**: Public‐health resources (education, clinics, supply chains) can be allocated cluster-wise.
  * **Monitoring**: Over time, you can track whether counties move between clusters—e.g., a county migrating from a **low-uptake** cluster into a high-uptake one signals successful policy.


## Anomaly/Outlier Detection

In [ ]:
import numpy as np
from IPython.display import display

In [ ]:
# Then, we normalize column names
df.columns = (
    df.columns.str.strip()
                .str.lower()
                .str.replace(' ', '_', regex=False)
                .str.replace('/', '_', regex=False)
)

# Next, we identify coverage-rate columns (values between 0 and 1)
cov_cols = [
    c for c in df.select_dtypes(include=[np.number]).columns
    if df[c].dropna().between(0,1).all()
]

# Then, we ensure 'county' and 'school_year' exist
if not {'county', 'school_year'}.issubset(df.columns):
    raise KeyError("Expected 'county' and 'school_year' columns in the data.")

# Next, we aggregate by county & school_year
grouped = df.groupby(['county', 'school_year'], as_index=False)[cov_cols].mean()

# Then, we detect anomalies via IQR on year-over-year diffs
anomalies = []
for vaccine in cov_cols:
    pivot = grouped.pivot(index='county', columns='school_year', values=vaccine)
    pivot = pivot.reindex(sorted(pivot.columns), axis=1)
    diffs = pivot.diff(axis=1)
    s = diffs.stack(dropna=True)
    if s.empty:
        continue
    Q1, Q3 = s.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    outliers = s[(s < lower) | (s > upper)]
    for (county, year), change in outliers.items():
        anomalies.append({
            'county': county,
            'year': year,
            'vaccine': vaccine,
            'change': change
        })

# Finally, we compile and display results
if not anomalies:
    print("No significant coverage spikes or drops detected at the county level.")
else:
    anomalies_df = pd.DataFrame(anomalies)
    anomalies_df['abs_change'] = anomalies_df['change'].abs()
    anomalies_df = anomalies_df.sort_values('abs_change', ascending=False)
    print("Top coverage spikes/drops by magnitude:")
    display(anomalies_df.head(20))

**Aggregation**  
  * We groups the data by county and school year, taking the mean coverage for each vaccine in each county–year combination.

**Year-Over-Year Differences**  
  * For each **vaccine**, pivots the table so that each column is one school year.
  * **Computes** the difference between consecutive years for every county (e.g. **2020–2021** minus **2019–2020**).

**Outlier Detection via IQR**  
  * Treats the distribution of all those year-over-year changes as a sample, and calculates the interquartile range (IQR).
  * Flags any change that lies below **Q1 − 1.5 × IQR** or above **Q3 + 1.5 × IQR** as an anomaly—i.e. an unusually large drop or spike in coverage.


## Predictive Modeling & Forecasting

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# We normalize column names
df.columns = (
    df.columns.str.strip()
                .str.lower()
                .str.replace(' ', '_', regex=False)
                .str.replace('/', '_', regex=False)
)

# Then, we identify coverage-rate columns (values between 0 and 1)
cov_cols = [
    c for c in df.select_dtypes(include=['float64','int64']).columns
    if df[c].dropna().between(0,1).all()
]

# Next, we compute yearly mean coverage
yearly = df.groupby('school_year', as_index=False)[cov_cols].mean()

# Then, we figure out next school-year label
def next_year_label(label):
    start, end = label.split('-')
    return f"{end}-{int(end)+1}"
last = yearly['school_year'].iloc[-1]
forecast_label = next_year_label(last)

# Next, we fit ARIMA per vaccine and forecast
for vaccine in cov_cols:
    series = yearly[vaccine].astype(float)
    model = ARIMA(series, order=(1,1,1))
    fit = model.fit()
    fc = fit.get_forecast(steps=1).summary_frame()  # DataFrame with mean & CIs
    
    yhat   = fc['mean'].iloc[0]
    lower  = fc['mean_ci_lower'].iloc[0]
    upper  = fc['mean_ci_upper'].iloc[0]
    
    print(f"{vaccine.upper()} forecast for {forecast_label}: "
          f"{yhat:.3f} (95% CI: {lower:.3f}–{upper:.3f})")
    
    # Plot history + forecast
    plt.figure(figsize=(6,4))
    plt.plot(yearly['school_year'], series, marker='o', label='History')
    plt.scatter([forecast_label], [yhat], color='red', marker='X', s=100, label='Forecast')
    plt.fill_between(
        [forecast_label], [lower], [upper],
        color='pink', alpha=0.3, label='95% CI'
    )
    plt.title(f"ARIMA(1,1,1) Forecast – {vaccine.title()}")
    plt.xlabel('School Year')
    plt.ylabel('Coverage Rate')
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

**Aggregate by Year**  
  * Group the data by **school_year** and take the mean of each vaccine’s coverage that year.
  * This produces a table with one row per school year and one column per vaccine.

**Define the Forecast Horizon**  
  * Look at the last year label (e.g. **"2023-2024"**) and compute the next one (e.g. **"2024-2025"**) so we know how to label our forecast point.

**Fit an ARIMA(1,1,1) Model**  
  * For each vaccine’s time series (a 5-point sequence of yearly coverage rates), we fit a simple **ARIMA(1,1,1)** model:
    * **AR(1)**: the forecast depends on the previous year’s value.
    * **I(1)**: we difference once to remove any linear trend.
    * **MA(1)**: the forecast error depends on the error from the prior year.
    * **ARIMA** is a classical time-series approach that balances fidelity to the data with smoothing.

**Generate the One-Year Forecast**  
  * We ask the model to predict one step ahead (the next school year).
  * **We extract**:
    * **mean (yhat)**: the point estimate for next year’s coverage.
    * **mean_ci_lower** and **mean_ci_upper**: the bounds of the **95% confidence interval** around that estimate.

**Visualization**    
  * We plot the historical coverage as markers connected by lines.
  * We then overlay the forecast point (a red “X”) at the **next-year** label, with a shaded ribbon showing the **95% CI**.
  * This visual helps you see how the forecast relates to past trends and how wide the uncertainty is.

## Threshold & Compliance Monitoring

In [ ]:
# Here, we identify the ISD identifier column
isd_col = "school_type"  
if isd_col not in df.columns:
    # Fallback: try common alternatives
    for alt in ["district", "school_district", "isd"]:
        if alt in df.columns:
            isd_col = alt
            break
    else:
        raise KeyError("Could not find an ISD/district column in your data.")

# Then, we identify coverage‐rate columns (values between 0 and 1)
coverage_cols = [
    c for c in df.select_dtypes(include=["float64","int64"]).columns
    if df[c].dropna().between(0,1).all()
]

# Next, we define your public‐health thresholds here
thresholds = {
    "mmr": 0.95,
    "tdap/td": 0.90,
    "meningococcal": 0.90,
    "hepatitis_a": 0.85,
    "hepatitis_b": 0.85,
    "polio": 0.95,
    "varicella": 0.95,
}

# Then, we tabulate how many unique ISDs fall below each threshold
results = []
for vac, thresh in thresholds.items():
    if vac not in coverage_cols:
        continue
    below = df[df[vac] < thresh]
    count = below[isd_col].nunique()
    results.append({"Vaccine": vac, "Threshold": thresh, "ISDs Below": count})

result_df = pd.DataFrame(results).sort_values("ISDs Below", ascending=False)
print("Number of ISDs below each coverage threshold:\n")
print(result_df.to_string(index=False))

In [ ]:
# Here, we identify identifier and school_type columns
id_cols = [c for c in df.columns if any(key in c.lower() for key in ["county", "facility_name"])]
id_col = id_cols[0] if id_cols else None
school_type_col = "school_type" if "school_type" in df.columns else None

if not id_col:
    raise KeyError("No ISD/district identifier column found.")
if not school_type_col:
    raise KeyError("No 'school_type' column found.")

# Then, we define critical thresholds
thresholds = {
    "mmr": 0.95,
    "tdap/td": 0.90,
    "meningococcal": 0.90,
    "hepatitis_a": 0.85,
    "hepatitis_b": 0.85,
    "polio": 0.95,
    "varicella": 0.95,
}

# Next, we coerce coverage columns to numeric
for vac in thresholds:
    if vac in df.columns:
        df[vac] = pd.to_numeric(df[vac], errors='coerce')

# Then, we automatically flag schools below threshold
flags = []
for vac, thresh in thresholds.items():
    if vac in df.columns:
        flagged = df[df[vac] < thresh].copy()
        if not flagged.empty:
            flagged = flagged[[id_col, school_type_col, "school_year", vac]]
            flagged["Vaccine"] = vac
            flagged["Coverage"] = flagged[vac]
            flagged["Threshold"] = thresh
            flags.append(flagged[[id_col, school_type_col, "school_year", "Vaccine", "Coverage", "Threshold"]])

# Here, we compile, sort by Threshold, and display flagged schools
if flags:
    flags_df = pd.concat(flags, ignore_index=True)
    flags_df = flags_df.sort_values("Threshold")
    display(flags_df)
else:
    print("No schools currently fall below the critical thresholds.")